In [ ]:
import numpy as np
from bayes_opt import BayesianOptimization

data = np.load('../game_results/game_results_humans.npz')
xyz = data['xyz']
params = data['params']
meta = data['meta']

# Ground truth model in numpy
def gmm_model(params, x, y):
    # params: (N, 25), x/y: (N, 20)
    g = params[:24].reshape(6, 4)
    w, mx, my, s = g[:, 0], g[:,1], g[:, 2], g[:, 3]
    expo = -((x[...,None] - mx[None])**2 + (y[...,None] - my[None])**2) / s[None]
    res = np.sum(w[None] * np.exp(expo), axis=-1)
    
    return res * params[24]


# for i in range(len(params)):
#     assert np.allclose(gmm_model(params[i], xyz[i,:,0], xyz[i,:,1]), xyz[i,:,2])

In [ ]:
# import numpy as np
# from skopt.space import Integer
# import skopt.space
# from skopt import gp_minimize
# from skopt.utils import use_named_args

# # grid
# x = np.linspace(0,1,100,endpoint=False)
# grid = np.meshgrid(x,x)
# search_space = [skopt.space.Integer(0, 99, name='x'), skopt.space.Integer(0, 99, name='y')]
# #objective function
# def f_objective(point):
#     x,y=point
#     return objective[x,y]

# for person in xyz[:2]:
#     for i,trial in enumerate(person):
#         x0=[[int(x0[0]*100),int(x0[1]*100)] for x0 in xyz[i][:4]]

#         for k,point in enumerate(trial):
#             objective=-gmm_model(params[k], grid[0], grid[1])
#             res=gp_minimize(f_objective, search_space, n_calls=20, 
#                             x0=[[int(xy[0]*100),int(xy[1]*100)] for xy in trial[:len(x0)]],acq_func='EI')
#             print(res['x_iters'])
#             x0.append(res['x_iters'][-1])

In [ ]:
# import numpy as np
# x = np.linspace(0,1,100,endpoint=False)
# grid = np.meshgrid(x,x)
# def f_objective(point):
#     x,y=point
#     return objective[x,y]


# from skopt.space import Integer
# import skopt.space
# from skopt import gp_minimize
# from skopt.utils import use_named_args
# search_space = [skopt.space.Integer(0, 99, name='x'), skopt.space.Integer(0, 99, name='y')]
# for i in range(2,3):
#     objective=-gmm_model(params[i], grid[0], grid[1])
#     res=gp_minimize(f_objective, search_space, n_calls=20,n_initial_points=0, 
#                     x0=[[int(x0[0]*100),int(x0[1]*100)] for x0 in xyz[i][:4]],acq_func='EI',
#                     acq_optimizer='auto')


# print(len(res['models']))
# print(len(res['x_iters']))
# print(len(res['func_vals']))
# res

In [23]:
# print(res['models'][0])

In [2]:
import sys
from pathlib import Path
import numpy as np

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from BayesianOptimization.BO_functions import probability_of_improvement, expected_improvement, GP_UCB_original, squared_exponential_kernel, fit_predictive_GP, optimize_GP_hyperparams

c:\Users\julie\Desktop\Uni\semester_4\Active machine learning and agency\Blok 1 - Bayesian Optimization\Group-24-Bayesian-Optimization\.venv\Lib\site-packages\torch\__init__.py:1305: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)


In [ ]:
np.random.seed(32)
# points that we are considering
X = np.linspace(0,1,100,endpoint=False)
grid = np.meshgrid(X,X)
xx, yy = grid
Xtest = np.column_stack([xx.ravel(), yy.ravel()])

def f_objective(point):
    x,y=point
    x = min(99,int(x*100))
    y = min(99, int(y*100))
    return objective[x,y]


# prior over the lengthscale
prior_mean = 0 
prior_std = 5

# trade-off hyperparams
eps = 0.5
    
for i, person in enumerate(xyz[:2]):
    x0=[[x0[0],x0[1]] for x0 in person[:4]]
    objective = -gmm_model(params[i], grid[0], grid[1])            
    X_sample = x0
    y_sample = [f_objective((xy[0],xy[1])) for xy in x0]
    current_best = np.max(y_sample)

    for k, point in enumerate(person[3:19]):
        lengthscale, output_variance, noise_variance = optimize_GP_hyperparams(X_sample, y_sample, 500, 5e-3, prior_mean, prior_std)
        mu, covariance = fit_predictive_GP(X_sample, y_sample, Xtest, lengthscale, output_variance, noise_variance)
        std = np.sqrt(np.diag(covariance))

        acquisition_values_pi = probability_of_improvement(current_best,  mu.flatten(), std, eps)
        acquisition_values_ei = expected_improvement(current_best,  mu.flatten(), std, eps)
        acquisition_values_gp_ucb = GP_UCB_original(mu.flatten(), std, eps)

        X_sample.append(list(person[k+4][:2]))

        y_sample.append(person[k+4][2])

        current_best = np.max(y_sample)


(10000,)


KeyboardInterrupt: 